## Converts COCO-style hand keypoint annotations to CSV format 

In [1]:
import json
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image

def coco_to_csv(coco_json_path, images_dir, output_csv='keypoints_dataset.csv', label_name='gesture'):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    image_dict = {img['id']: img['file_name'] for img in coco_data['images']}
    
    rows = []
    for ann in tqdm(coco_data['annotations'], desc="Processing annotations"):
        image_id = ann['image_id']
        keypoints = ann['keypoints']
        file_name = image_dict[image_id]
        img_path = os.path.join(images_dir, file_name)

        # Get image size for normalization
        with Image.open(img_path) as img:
            w, h = img.size

        # Extract only x, y coords
        coords = []
        for i in range(0, len(keypoints), 3):
            x = keypoints[i] / w
            y = keypoints[i+1] / h
            coords.extend([x, y])

        label = ann.get('category_id', 0)

        rows.append(coords + [label])

    columns = [f"x{i}" if i%2==0 else f"y{i//2}" for i in range(42)] + [label_name]
    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"Saved {len(df)} samples to {output_csv}")
    return df


In [25]:
from pathlib import Path
print("CWD:", Path.cwd())
print("raw exists:", Path("../data/raw/hand_keypoint_dataset_26k/hand_keypoint_dataset_26k").resolve().exists())
print("anno exists:", Path("../data/raw/hand_keypoint_dataset_26k/hand_keypoint_dataset_26k/coco_annotation/").resolve().exists())
print("images exists:", Path("../data/raw/hand_keypoint_dataset_26k/hand_keypoint_dataset_26k/images/train").resolve().exists())
print("processed dir exists:", Path("../data/processed").resolve().exists())


CWD: c:\Users\flees\Desktop\Hand-Gesture-Recognition\notebooks
raw exists: True
anno exists: True
images exists: True
processed dir exists: True


In [27]:
coco_to_csv(
    coco_json_path='../data/raw/hand_keypoint_dataset_26k/hand_keypoint_dataset_26k/coco_annotation/train/_annotations.coco.json',
    images_dir='../data/raw/hand_keypoint_dataset_26k/hand_keypoint_dataset_26k/images/train',
    output_csv='../data/processed/train_keypoints.csv'
)

Processing annotations: 100%|██████████| 18776/18776 [00:01<00:00, 10693.27it/s]


OSError: [Errno 28] No space left on device

In [ ]:
coco_to_csv(
    coco_json_path='../data/raw/hand_keypoint_dataset_26k/hand_keypoint_dataset_26k/coco_annotation/val/_annotations.coco.json',
    images_dir='../data/raw/hand_keypoint_dataset_26k/hand_keypoint_dataset_26k/images/val',
    output_csv='../data/preprocessed/val_keypoints.csv'
)
